<a href="https://colab.research.google.com/github/AkhilVinayakp/HAR_LSTM_CNN/blob/main/deepStacked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
!pip install deepstack

In [4]:
%cd /content/drive/My\ Drive/datasets 
%pwd

/content/drive/My Drive/datasets


'/content/drive/My Drive/datasets'

In [19]:
df = pd.read_csv('int_norm.csv')

In [20]:
def create_windowed_dataset(series, window_size = 128,shift_size=64, batch_size=None,shuffle_buffer=None):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size , shift = shift_size, drop_remainder = True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size))
  dataset = dataset.map(lambda window: ( window[:,1:], tf.one_hot(tf.cast(window[window_size -1 , 0], tf.int64), depth=6) ))
  dataset = dataset.shuffle(192)
  return dataset

In [21]:
df.drop('timestamp', axis = 1, inplace=True)

In [22]:
df

,user,activity,x-axis,y-axis,z-axis
0,33,Jogging,-0.054655,0.997718,0.039652
1,33,Jogging,0.405338,0.910910,0.077102
2,33,Jogging,0.410782,0.911708,-0.006846
3,33,Jogging,-0.032685,0.986373,0.161248
4,33,Jogging,-0.083804,0.856341,0.509566
...,...,...,...,...,...
1098199,19,Sitting,0.968689,-0.168982,0.181898
1098200,19,Sitting,0.970048,-0.156667,0.185640
1098201,19,Sitting,0.972323,-0.147776,0.180972
1098202,19,Sitting,0.969793,-0.157322,0.186416


In [23]:
df['activity'].unique()

array(['Jogging', 'Walking', 'Upstairs', 'Downstairs', 'Sitting',
       'Standing'], dtype=object)

In [24]:
from sklearn.preprocessing import LabelEncoder

In [25]:
enc = LabelEncoder()
df['activity'] = enc.fit_transform(df['activity'])

In [26]:
df['activity'].unique()

array([1, 5, 4, 0, 2, 3])

In [27]:
enc.inverse_transform(df['activity'].unique())

array(['Jogging', 'Walking', 'Upstairs', 'Downstairs', 'Sitting',
       'Standing'], dtype=object)

In [34]:
# spliting the dataset according to          
ls = []
for i in df['activity'].unique():
  s_df = df[df['activity'] == i]
  print(f"{i} enc {enc.inverse_transform([i])}")
  ls.append(create_windowed_dataset(s_df))

1 enc ['Jogging']
5 enc ['Walking']
4 enc ['Upstairs']
0 enc ['Downstairs']
2 enc ['Sitting']
3 enc ['Standing']


In [36]:
jog_set = ls[1]
walk_set = ls[5]
upst_set = ls[4]
dost_set = ls[0]
sit_set = ls[2]
std_set = ls[3]

In [49]:
jog_set_f = []
jog_set_l = []
for i in jog_set.as_numpy_iterator():
  jog_set_f.append(i[0])
  jog_set_l.append(i[1])

# converting to numpy array
jog_set_f = np.array(jog_set_f)
jog_set_l = np.array(jog_set_l)
# shape of each
print(f"train features {jog_set_f.shape}")
print(f"train labels {jog_set_l.shape}")
jog_train = (jog_set_f[:5000],jog_set_l[:5000])
jog_test = (jog_set_f[5000:], jog_set_l[5000:])
print(f"jog train{len(jog_train)} jog testing {len(jog_test)}")

train features (6630, 128, 4)
train labels (6630, 6)
jog train2 jog testing 2


In [50]:
walk_set_f = []
walk_set_l = []
for i in walk_set.as_numpy_iterator():
  walk_set_f.append(i[0])
  walk_set_l.append(i[1])

# converting to numpy array
walk_set_f = np.array(walk_set_f)
walk_set_l = np.array(walk_set_l)
# shape of each
print(f"train features {walk_set_f.shape}")
print(f"train labels {walk_set_l.shape}")
walk_train = (walk_set_f[:500],walk_set_l[:500])
walk_test = (walk_set_f[500:], walk_set_l[500:])

train features (755, 128, 4)
train labels (755, 6)


In [51]:
dost_set_f = []
dost_set_l = []
for i in dost_set.as_numpy_iterator():
  dost_set_f.append(i[0])
  dost_set_l.append(i[1])

# converting to numpy array
dost_set_f = np.array(dost_set_f)
dost_set_l = np.array(dost_set_l)
# shape of each
print(f"train features {dost_set_f.shape}")
print(f"train labels {dost_set_l.shape}")
dost_train = (dost_set_f[:4000],dost_set_l[:4000])
dost_test = (dost_set_f[4000:], dost_set_l[4000:])

train features (5345, 128, 4)
train labels (5345, 6)


In [52]:
upst_set_f = []
upst_set_l = []
for i in upst_set.as_numpy_iterator():
  upst_set_f.append(i[0])
  upst_set_l.append(i[1])

# converting to numpy array
upst_set_f = np.array(upst_set_f)
upst_set_l = np.array(upst_set_l)
# shape of each
print(f"train features {upst_set_f.shape}")
print(f"train labels {upst_set_l.shape}")
upst_train = (upst_set_f[:600],upst_set_l[:600])
upst_test = (upst_set_f[600:], upst_set_l[600:])

train features (935, 128, 4)
train labels (935, 6)


In [53]:
sit_set_f = []
sit_set_l = []
for i in sit_set.as_numpy_iterator():
  sit_set_f.append(i[0])
  sit_set_l.append(i[1])

# converting to numpy array
sit_set_f = np.array(sit_set_f)
sit_set_l = np.array(sit_set_l)
# shape of each
print(f"train features {sit_set_f.shape}")
print(f"train labels {sit_set_l.shape}")
sit_train = (sit_set_f[:900],sit_set_l[:900])
sit_test = (sit_set_f[900:], sit_set_l[900:])

train features (1918, 128, 4)
train labels (1918, 6)


In [54]:
std_set_f = []
std_set_l = []
for i in std_set.as_numpy_iterator():
  std_set_f.append(i[0])
  std_set_l.append(i[1])

# converting to numpy array
std_set_f = np.array(sit_set_f)
std_set_l = np.array(sit_set_l)
# shape of each
print(f"train features {std_set_f.shape}")
print(f"train labels {std_set_l.shape}")

std_train = (std_set_f[:900],std_set_l[:900])
std_test = (std_set_f[900:], std_set_l[900:])

train features (1918, 128, 4)
train labels (1918, 6)


In [55]:
# # adding globel average pooling to the model
# codding lstm layers and cnn
model_bi_dir = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: x,
                      input_shape=[128, 3]),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
  tf.keras.layers.SpatialDropout1D(0.4),
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)),
  tf.keras.layers.Conv2D(64,5,strides=(2,2)),
  tf.keras.layers.MaxPooling2D((2, 2), strides = (2,2)),
  
  tf.keras.layers.Conv2D(128,3,strides=(1,1)),  
  tf.keras.layers.GlobalAveragePooling2D(),
  # tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)),
  # parameters ?
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(6, activation='softmax')
  
])

In [56]:
from deepstack.base import KerasMember  # For a generic (i.e., Non-Keras Model) check the class `Member`
from deepstack.ensemble import StackEnsemble

In [58]:
model_jog = model_bi_dir  
train_batches_jog = jog_train   
val_batches_jog = jog_test
member2 = KerasMember(name="Model_jog", keras_model=model_jog, train_batches=jog_train, val_batches=jog_test)


ValueError: ignored